In [51]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2023-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

In [52]:
Grass_womens=hist_data_dog('Grass','Womens',1,2,datefilter)
Grass_mens=hist_data_dog('Grass','Mens',1,2,datefilter)
temp=pd.concat([Grass_womens,Grass_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()


In [53]:
import pandas as pd
calc=temp.copy()
# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the next 30 days after the match (excludes current match)
def wins_last_30_after_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened after the cutoff date within the next 30 days
        future_wins = group[(group['Date'] > cutoff) & (group['Date'] <= cutoff + pd.Timedelta(days=30))]
        result.append(len(future_wins))  # Count the wins in the next 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the next 30 days after each match
player_wins['Wins_Last_30_Days_After'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_after_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "after match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (after the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_After']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_After': 'Fav_Wins_Last_30_Days_After'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='forward'
).drop(columns=['Match_Date'])

# Merge back for Dog (after the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_After']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_After': 'Dog_Wins_Last_30_Days_After'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='forward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no future wins)
calc['Fav_Wins_Last_30_Days_After'] = calc['Fav_Wins_Last_30_Days_After'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_After'] = calc['Dog_Wins_Last_30_Days_After'].fillna(0).astype(int)

Fav_Win_Count=calc[['Date','Fav','Fav_Wins_Last_30_Days_After']].copy()
Fav_Win_Count['Date'] = pd.to_datetime(Fav_Win_Count['Date'])
latest_per_fav = Fav_Win_Count[Fav_Win_Count['Date'] == Fav_Win_Count.groupby('Fav')['Date'].transform('max')]
latest_per_fav[['Fav','Fav_Wins_Last_30_Days_After']].to_csv('Fav_Win_Count_Grass.csv')
Dog_Win_Count=calc[['Date','Dog','Dog_Wins_Last_30_Days_After']].copy()
Dog_Win_Count['Date'] = pd.to_datetime(Dog_Win_Count['Date'])
latest_per_dog = Dog_Win_Count[Dog_Win_Count['Date'] == Dog_Win_Count.groupby('Dog')['Date'].transform('max')]
latest_per_dog[['Dog','Dog_Wins_Last_30_Days_After']].to_csv('Dog_Win_Count_Grass.csv')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5344\908341263.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_after_match)


In [54]:
calc=temp.copy()
import pandas as pd

# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the last 30 days, excluding the current match
def wins_last_30_before_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened before the cutoff date within the last 30 days
        past_wins = group[(group['Date'] < cutoff) & (group['Date'] >= cutoff - pd.Timedelta(days=30))]
        result.append(len(past_wins))  # Count the wins in the past 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the last 30 days for each player before the match
player_wins['Wins_Last_30_Days_Before'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_before_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "before match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (before the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Fav_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Merge back for Dog (before the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Dog_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no prior wins)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_5344\74366135.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_before_match)


In [55]:
calc['Predicted_Winner'] = calc['Fav_Wins_Last_30_Days_Before'] < calc['Dog_Wins_Last_30_Days_Before']
accuracy = (calc['Predicted_Winner'] == calc['Dog_Win']).mean()
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 66.79%


In [56]:
stake=100
date_filter=calc[calc['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.4].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter.to_csv('_GrassResults_Womens.csv')

Total Profit: 728.9999999999999 30 0.5333333333333333


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
159,2025-06-12,2.32,Veronika Kudermetova,Greet Minnen,Veronika Kudermetova,1,0,2,0
162,2025-06-12,2.59,Bianca Andreescu,Lulu Sun,Bianca Andreescu,2,0,1,0
164,2025-06-13,2.56,Rebecca Marino,Rebecca Marino,Alexandra Eala,1,1,1,0
174,2025-06-14,1.90,Rebecca Marino,Rebecca Marino,Celine Naef,2,1,2,0
175,2025-06-14,2.56,Gabriela Ruse Elena,Elisabetta Cocciaretto,Gabriela Ruse Elena,1,0,1,0
219,2025-06-16,2.30,Alexandra Eala,Alexandra Eala,Alexia Todoni Anca,1,1,1,0
233,2025-06-17,1.99,Magda Linette,Alexandra Eala,Magda Linette,1,1,1,0
238,2025-06-17,2.54,Xin Yu Wang,Xin Yu Wang,Daria Kasatkina,2,3,2,0
243,2025-06-18,2.42,Diana Shnaider,Donna Vekic,Diana Shnaider,2,0,2,1
290,2025-06-21,2.53,Xin Yu Wang,Xin Yu Wang,Liudmila Samsonova,3,1,3,1


In [57]:
stake=100
date_filter=calc[calc['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter.to_csv('_GrassResults_Mens.csv')

Total Profit: 107.99999999999996 9 0.5555555555555556


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
102,2025-06-02,1.93,Zachary Svajda,Li Tu,Zachary Svajda,2,0,1,0
103,2025-06-03,2.02,Rinky Hijikata,Billy Harris,Rinky Hijikata,2,0,2,0
204,2025-06-15,1.96,Yannick Hanfmann,Yannick Hanfmann,Nicolas Jarry,1,1,1,0
224,2025-06-17,2.01,Mackenzie Mcdonald,Mackenzie Mcdonald,Gael Monfils,1,1,3,0
274,2025-06-20,2.10,Arthur Rinderknech,Arthur Rinderknech,Reilly Opelka,3,2,2,0
374,2025-06-26,2.02,Jenson Brooksby,Daniel Evans,Jenson Brooksby,5,0,4,0
384,2025-06-27,1.90,Alex Michelsen,Alex Michelsen,Roberto Bautista Agut,1,1,3,2
418,2025-06-30,1.93,Cameron Norrie,Roberto Bautista Agut,Cameron Norrie,2,1,2,0
522,2025-07-04,2.11,Luciano Darderi,Luciano Darderi,Arthur Fery,2,2,1,0


In [58]:
#from sqlalchemy import text

#with devengine.connect() as conn:
#    conn.execute(text('DELETE FROM results_Grass_1 WHERE Date > "2025-07-04"'))
#    conn.commit()
#pd.read_sql_query('Select distinct* from AllMatches where date = "2025-06-08" and Sex like "Womens"',con=devengine)

In [59]:
date_filter = pd.read_pickle(r'.\Grass_Today')#.drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_win = pd.read_csv(r'.\Fav_Win_Count_Grass.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count_Grass.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_win_subset = fav_win[['Fav', 'Fav_Wins_Last_30_Days_After']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Wins_Last_30_Days_After']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_win_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
today=date_filter.copy()
today=today[today['Fav_Odds']>1.4]
today=today[today['Dog_Wins_Last_30_Days_After']>today['Fav_Wins_Last_30_Days_After']]
womens=today[today['Sex']=='Womens']
womens[['Time','Dog','Dog_Odds','Fav']]

,Time,Dog,Dog_Odds,Fav
1,00:55,Barbora Krejcikova,2.73,Emma Navarro
10,23:30,Sonay Kartal,1.92,Anastasia Pavlyuchenkova


In [60]:
date_filter = pd.read_pickle(r'.\Grass_Today')#.drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_win = pd.read_csv(r'.\Fav_Win_Count_Grass.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count_Grass.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_win_subset = fav_win[['Fav', 'Fav_Wins_Last_30_Days_After']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Wins_Last_30_Days_After']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_win_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
today=date_filter.copy()
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days_After']>today['Fav_Wins_Last_30_Days_After']]
mens=today[today['Sex']!='Womens']
mens[['Time','Dog','Dog_Odds','Fav','Fav_Odds']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds


In [61]:
# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)  
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

mens['Date']=date_string
mens.to_sql('Preditions_Grass',con=devengine,if_exists='append',index=False)

womens['Date']=date_string
womens.to_sql('Preditions_Grass',con=devengine,if_exists='append',index=False)

2

In [62]:
predictions=pd.read_sql_query('Select Distinct * from Preditions_Grass where date > "2025-07-04"',con=devengine)
results=temp
stake=100

# Ensure date types
predictions['Date'] = pd.to_datetime(predictions['Date'])
results['Date'] = pd.to_datetime(results['Date'])

# Step 1: Initial merge on the exact date
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],
    on=['Date', 'Fav', 'Dog'], 
    how='left'
)

# Step 2: Get unmatched rows only
unmatched = merged_df[merged_df['Winner'].isna()].copy()

if not unmatched.empty:
    # Try matching with results where match happened one day after the prediction date
    results_shifted = results.copy()
    results_shifted['Date'] = results_shifted['Date'] - pd.Timedelta(days=1)

    fallback = pd.merge(
        unmatched.drop(columns=['Winner']),  # remove old (NaN) winner
        results_shifted[['Date', 'Fav', 'Dog', 'Winner']],
        on=['Date', 'Fav', 'Dog'],
        how='left'
    )

    # Only update original merged_df where Winner was missing AND fallback found a match
    merged_df.loc[merged_df['Winner'].isna(), 'Winner'] = fallback['Winner'].values


# merged_df now has best effort match: same day, or next day fallback
merged_df_filtered=merged_df
#merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Sex']=='Womens']


merged_df_filtered['Profit'] = merged_df_filtered.apply(    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered[['Date','Dog_Odds','Winner','Dog','Fav','Profit']].head(10)


Total Profit: -200 2 0.0


,Date,Dog_Odds,Winner,Dog,Fav,Profit
0,2025-07-06,2.73,NaN,Barbora Krejcikova,Emma Navarro,-100
1,2025-07-06,1.92,NaN,Sonay Kartal,Anastasia Pavlyuchenkova,-100


In [63]:
#df=pd.read_clipboard()
#df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
#df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
#df.to_sql('Preditions_Grass',con=devengine,if_exists='replace',index=False)



#pd.read_sql_query('Select * from Preditions_Grass',con=devengine).to_clipboard(index=False)